<a href="https://colab.research.google.com/github/smeenakshi1997/GRIHAM-C_OGD/blob/main/GRIHAMC_TrainingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Topic Clustering Model Training using subset of the given dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers #==4.37.2
!pip install torch #==2.1.0+cu121 #==2.2.0
!pip install tqdm
!pip install scikit-learn

In [ ]:
#!pip install tqdm

In [ ]:
#print the version of transformers and torch
import transformers
import torch
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)


Transformers version: 4.37.2
Torch version: 2.1.0+cu121


In [ ]:
import pandas as pd

df_excel = pd.read_excel('/content/problem_statement_1_and_2/new_file_Description_stages.xlsx')

print(df_excel)

In [ ]:

# Example DataFrame
data = df_excel
df = pd.DataFrame(data)

# Find unique values in the "Description" column
unique_values = df['stage_0.2'].unique()

# Print the unique values
print("Unique Values in Description column:", unique_values)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Example DataFrame
data = df_excel
df = pd.DataFrame(data)

# Use LabelEncoder to encode ministry names
#label_encoder = LabelEncoder()
#df['numeric_label'] = label_encoder.fit_transform(df['Description'])

# Create dynamic entailment mapping based on unique ministries
unique_ministries = df['stage_0.2'].unique()
entailment_mapping = {}

for text, ministry in zip(df['text_column'], df['stage_0.2']):
    # Assign entailment values based on unique ministry
    text = text.replace("\r\n", ' ')
    entailment_mapping[text] = {'ministry': ministry, 'entailment': 1}

# Apply the mapping to create numerical labels based on entailment
df['numeric_label'] = df['text_column'].map(lambda x: entailment_mapping[x]['entailment'])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BartTokenizer, BartForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx]['text_column'])
        label = int(self.dataframe.iloc[idx]['numeric_label'])

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli') #facebook/bart-large-mnli
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli') #facebook/bart-large-mnli

# Create custom datasets and dataloaders
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Training parameters
num_epochs = 5
learning_rate = 3e-5

# Initialize the optimizer and scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Loss function
loss_fn = CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Average validation loss: {avg_val_loss}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Average validation loss: 0.0018046391553070862


Epoch 5/5: 100%|██████████| 160/160 [03:55<00:00,  1.47s/it]


Average training loss: 0.0022183288801898016


Epoch 5/5 - Validation: 100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

Average validation loss: 0.0017562266410095617




# Accuracy Predicted

In [ ]:
# Example of calculating accuracy
correct_predictions = (torch.argmax(outputs.logits, dim=1) == labels).sum().item()
accuracy = correct_predictions / labels.size(0)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [ ]:
# Save the fine-tuned model
model.save_pretrained('/content/problem_statement_1_and_2/cluster_model_worksfine')
tokenizer.save_pretrained('/content/problem_statement_1_and_2/cluster_model_worksfine')

In [ ]:
#Run this cell to save the pickle file to run in GPU
import pickle
model_data = {'model': model, 'tokenizer': tokenizer}

pickle_file_path = '/content/problem_statement_1_and_2/cluster_model_worksfine.pkl'

with open(pickle_file_path, 'wb') as file:
    pickle.dump(model_data, file)

In [ ]:
#import pickle
#model_data = {'model': model, 'tokenizer': tokenizer}

#with open('fine_tuned_model.pkl', 'wb') as file:
#    pickle.dump(model_data, file)

In [ ]:
# Save the fine-tuned model and tokenizer with torch.save
#torch.save(model.state_dict(), 'fine_tuned_model.pth')
#tokenizer.save_pretrained('./fine_tuned_model')

# Load the fine-tuned model and tokenizer
#loaded_model = TFBartForSequenceClassification.from_pretrained('fine_tuned_model', state_dict=torch.load('fine_tuned_model.pth'))
#loaded_tokenizer = BartTokenizer.from_pretrained('./fine_tuned_model')


## **Summarizer model training for report generation based on dataset**

In [ ]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torch.optim as optim

# Sample DataFrame (replace this with your actual data)
data = pd.read_excel('/content/problem_statement_1_and_2/summary_training_ogd.xlsx')
df = pd.DataFrame(data)

class CustomSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_source_length=1024, max_target_length=150):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        source_text = str(self.dataframe.iloc[idx]['Text'])
        target_text = str(self.dataframe.iloc[idx]['Summary'])

        encoding = self.tokenizer(
            source_text,
            max_length=self.max_source_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        labels = self.tokenizer(
            target_text,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )['input_ids']

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Create custom dataset and dataloaders
train_dataset = CustomSummarizationDataset(df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Training parameters
num_epochs = 3
learning_rate = 1e-5

# Initialize the optimizer and scheduler
optimizer = optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Use SGD optimizer #AdamW(model.parameters(), lr=learning_rate) #optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Use SGD optimizer

total_steps = len(train_dataloader) * num_epochs

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

# Save the fine-tuned model
model.save_pretrained('/content/bart_large_cnn_fine_tuned_summarizer')
tokenizer.save_pretrained('/content/bart_large_cnn_fine_tuned_summarizer')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

Epoch 2/3: 100%|██████████| 19/19 [00:08<00:00,  2.26it/s]


Average training loss: 9.385990594562731


Epoch 3/3: 100%|██████████| 19/19 [00:08<00:00,  2.24it/s]


Average training loss: 9.145769119262695


('/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer/tokenizer_config.json',
 '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer/special_tokens_map.json',
 '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer/vocab.json',
 '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer/merges.txt',
 '/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer/added_tokens.json')

## **Topic Clustering system using the created model**

# **The below code works with GPU**

In [ ]:
import pickle
import torch
from transformers import pipeline, TFBartForSequenceClassification, BartTokenizer

# Load the fine-tuned model and tokenizer
#loaded_model = TFBartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
#loaded_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')

# Load the fine-tuned model and tokenizer from the pickle file
with open('/content/problem_statement_1_and_2/fine_tuned_model_worksfine.pkl', 'rb') as file:
  saved_model_data = pickle.load(file)

# Map the model to a CPU device
loaded_model = saved_model_data['model']
loaded_tokenizer = saved_model_data['tokenizer']

loaded_model.to('cpu')

classifier = pipeline("zero-shot-classification", model=loaded_model, tokenizer=loaded_tokenizer)

In [ ]:
def classification (text, stagelabels):
    top5 = []
    candidate_labels = stagelabels #catdf['stage2'] #['bank', 'received', 'right', 'wrong']
    # Tokenize the input string
    tokenizer = loaded_tokenizer

    tokenized_input = tokenizer(text, return_tensors="pt")
    # Now `tokenized_input` contains the necessary tokens for the model
    # You can pass it through the loaded model

    output = classifier(text, candidate_labels, multi_label=True)

    if (len(output['labels'])) > 5:
       for m in range (0,4):
          if output['labels']:
             top5.append(output['labels'][m])
    elif (len(output['labels'])) <= 5 and (len(output['labels'])) != 0:
         top5 = output['labels']
    else:
         top5 = []
    return (output, output['labels'][0], top5)


In [ ]:
def classify_items(json_data, org_code, sequence_to_classify): # instead of org_code stage 3 description can be given
    # Convert JSON data to a list of dictionaries
    data = json.loads(json_data)

    if org_code.startswith("GOV"):
       data = data
       original_data = data.copy()
    else:
       # Filter data by OrgCode
       data = [item for item in data if item['OrgCode'] == org_code]
       original_data = data.copy()
       # data = [item for item in data if item['Parent'] == (item ['Code'] if item['Description'] == stage3_description)] i.e. find the code of the stage3 item description

    #print(data)

    # Initialize result dictionary
    result = {}

    text = sequence_to_classify

    for stage in range(1, 9): # Alter the stage no according to the stage 3 description
        # Get items for the current stage
        stage_items = [item for item in data if item['Stage'] == stage]
        #print(stage_items)

        # If there are no items for the current stage, break the loop
        if not stage_items:
            break

        # Get the descriptions of the items
        descriptions = [item['Description'] for item in stage_items] #DescriptionHindi

        # Classify the descriptions
        output, best_item, top_5_items = classification(text, descriptions)

        # Add the best item to the result
        result[f'Stage {stage}'] = {
            'Best Item': best_item,
            'Top 5 Items': top_5_items
        }
        print(best_item)
        with open('result.json', 'w') as f:
             json.dump(result, f)

        for item in data:
            if item['Description'] == best_item:
               best_item_code = item['Code']
               break

        # Update data to only include items whose parent is the code of the best item
        data = [item for item in original_data if item['Parent'] == best_item_code]
        #print(data)
        #print(output)

    return result

In [ ]:
'''def classify_items(json_data, org_code, sequence_to_classify): # Hindi: instead of org_code stage 3 description can be given
    # Convert JSON data to a list of dictionaries
    data = json.loads(json_data)

    if org_code.startswith("GOV"):
       data = data
       original_data = data.copy()
    else:
       # Filter data by OrgCode
       data = [item for item in data if item['OrgCode'] == org_code]
       original_data = data.copy()
       # data = [item for item in data if item['Parent'] == (item ['Code'] if item['Description'] == stage3_description)] i.e. find the code of the stage3 item description

    #print(data)

    # Initialize result dictionary
    result = {}

    text = sequence_to_classify

    for stage in range(1, 9): # Alter the stage no according to the stage 3 description
        # Get items for the current stage
        stage_items = [item for item in data if item['Stage'] == stage]
        #print(stage_items)

        # If there are no items for the current stage, break the loop
        if not stage_items:
            break

        # Get the descriptions of the items
        descriptions = [item['DescriptionHindi'] for item in stage_items] #DescriptionHindi

        # Classify the descriptions
        output, best_item, top_5_items = classification(text, descriptions)

        # Add the best item to the result
        result[f'Stage {stage}'] = {
            'Best Item': best_item,
            'Top 5 Items': top_5_items
        }
        print(best_item)
        with open('result.json', 'w') as f:
             json.dump(result, f)

        for item in data:
            if item['DescriptionHindi'] == best_item:
               best_item_code = item['Code']
               break
        print(best_item_code)
        # Update data to only include items whose parent is the code of the best item
        data = [item for item in original_data if item['Parent'] == best_item_code]
        print(data)
        print(output)

    return result'''

In [ ]:
import json

with open('/content/problem_statement_1_and_2/no_pii_grievance.json', 'r') as f:
     grievance_df = json.load(f)

with open('/content/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'r') as c:
     # Load JSON data from file
     json_dat = json.load(c)    # Your JSON data

json_data = json.dumps(json_dat)
result = []

gdf = {"code":[], "text":[]}

gr_id =  input("Enter the Grievance Id: ")#"MEAPD/E/2023/0000002" #"GOVUP/E/2023/0000051" #"MEAPD/E/2023/0000002" #"CBODT/E/2023/0000001" #"MOLBR/E/2023/0000004" #"MORLY/E/2023/0000001" #"MOLBR/E/2023/0000003"
for id in grievance_df:
    if id['_id'] == gr_id:
       gdf["code"] = id['org_code']
       gdf["text"] = id['subject_content_text']
       break

#id = 0 #Id is the only needed to be given usually the unique id of the grievance.the id will fetch all the details from the DB including org code

sequence_to_classify = gdf['text'] #Text to classify
org_code = gdf['code'] #OrgCode

print(sequence_to_classify)


In [ ]:
result = classify_items(json_data, org_code, sequence_to_classify)

print(result)

In [ ]:
print(sequence_to_classify)
print(org_code)

In [ ]:
'''# prompt: rewrite a json file by adding a key value pair "DescriptionHindi": "" in the json objects which do not have the key value pair. The objects with the key value pair can be skipped.

import json

def add_description_hindi(json_data):
  """
  Adds a "DescriptionHindi" key with an empty string value to JSON objects that do not have it.

  Args:
    json_data: A list of JSON objects.

  Returns:
    A list of JSON objects with the "DescriptionHindi" key added.
  """

  updated_data = []
  for obj in json_data:
    if "Destination" not in obj:
      obj["Destination"] = ""
    updated_data.append(obj)

  return updated_data

# Load the JSON data from a file
with open('/content/drive/MyDrive/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'r') as f:
  json_data = json.load(f)

# Add the "DescriptionHindi" key
updated_data = add_description_hindi(json_data)

# Save the updated JSON data to a file
with open('/content/drive/MyDrive/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'w') as f:
  json.dump(updated_data, f)
'''

## **Summarizer system for Report Generation**



In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('/content/bart_large_cnn_fine_tuned_summarizer')
tokenizer = BartTokenizer.from_pretrained('/content/bart_large_cnn_fine_tuned_summarizer')

# Example text for summarization
input_text = sequence_to_classify
# Tokenize and generate summary
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(**inputs)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:", summary)

## **Additional Independent Translator for Indian Langauges (Except Odia): Use this cell to translate the sequence_to_classify for other language**

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

#article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

article = sequence_to_classify

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

lang_list = ["hi_IN", "ta_IN", "ml_IN" , "mr_IN", "bn_IN", "te_IN"]

tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

#sequence_to_classify = translation

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
#translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(translation)

#sequence_to_classify = translation

['The head of the United Nations says there is no military solution in Syria']
